#En este Notebook se implementa el código para tomar los audios de los sonidos respiratorios y dividirlos en nuevos audios teniendo encuenta los cicos respiratorios

La base de datos An open access database for the evaluation of respiratory sound classification algorithms se encuentra disponible en https://bhichallenge.med.auth.gr/

Link de descarga directa [Aqui](https://bhichallenge.med.auth.gr/sites/default/files/ICBHI_final_database/ICBHI_final_database.zip)

## Descargar e Importar librerias

In [ ]:
!pip install pysoundfile

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats
import math
import librosa as lb 
import soundfile as sf 

Para usar los archivos almacenados en Google Drive, se debe enlazar con la siguiente línea código

In [ ]:
from google.colab import drive
drive.mount(‘/content/gdrive’)

Sí descargo el archivo desde el enlace se debe descomprimir

In [ ]:
# Se descomprime los archivos
from zipfile import ZipFile
file_name = "ACA VA LA RUTA DONDE ESTA LA BASE DE DATOS" #aca se pone el archivo o la ruta
with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Completado")

Se definen los directorios de los archivos, estas rutas pueden cambiar según la ubicación de los archivos

In [ ]:

# El arvhivo de diagnóstico se encuentra en la base de datos con el nombre patien_diagnosis.csv
diagnosticos_csv = '/content/respiratory_sound_database/Respiratory_Sound_Database/patient_diagnosis.csv'
# Se define la ruta donde estan los archivos de audio en formato .wav
audio_text_loc = '/content/respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files'

In [ ]:
#se crea el directorio de salida, acá seran guardados los nuevos archivos
os.makedirs('salida2')

In [ ]:
# variable para longitud maxima de los audios
max_lon = 6

#Funciones usadas

In [ ]:
# Funcion para dividir el nombre del archivo 
def tokenize_file(filename):
    return filename.split('_')

# Función para dividir los archivos de audio
# Código tomado y modificado de https://pypi.org/project/audio-slice/

def dividir_audio(inicio, fin, raw_data,  sample_rate):
    max_ind = len(raw_data) 
    inicio_ind = min(int(inicio * sample_rate), max_ind)
    fin_ind = min(int(fin * sample_rate), max_ind)
    return raw_data[inicio_ind: fin_ind]

def compute_len(samp_rate=22050, time= 6, acquisition_mode=0):

    comp_len = 0
    if acquisition_mode == 1: #este se toma para sc (single channel) que se refiere a mono 
        comp_len = samp_rate * time
    else: #estereo o mc
        comp_len = (samp_rate * time) * 2

    return comp_len

#Tratamiento de los archivos

In [ ]:
#Se carga el archivo diagnosis.csv y semiran los encabezados
diagnosticos = pd.read_csv(diagnosticos_csv, names=['Id', 'diagnostico'])
diagnosticos.head()

In [ ]:
# Se revisa cuales son los diagnosticos que contiene la columna diagnostico
ds = diagnosticos['diagnostico'].unique()
ds

Se leen los archivos de la carpeta destino y se guardan en una lista llamada archivos, para esto se revisa la extesión.

In [ ]:
# Acá se van a obtener los archivos del directorio con extension .txt
archivos = [s.split('.')[0] for s in os.listdir(path = audio_text_loc) if '.txt' in s]
archivos 

Ahora con la lista de archivos, se lee cada uno para obtener la información, 
se hace un data frame (files_df) para guardar todos los datos y se guarda tambien como un csv

In [ ]:
#Se lee cada archivo, 
archivos_ = []
for f in archivos:
    df = pd.read_csv(audio_text_loc + '/' + f + '.txt', sep='\t', names=['inicio', 'fin', 'crackles', 'wheezes']) 
    df['archivo'] = f
    #Se divide cada archivo
    f_features = tokenize_file(f)
    df['Id'] = f_features[0]#df['pId'] = f_features[0]
    df['modo_adquisicion'] = f_features[3]
    archivos_.append(df)
    
archivos_df = pd.concat(archivos_)
archivos_df.reset_index()
#se imprime el encabezado del dataframe
archivos_df.head()

Se une la columna de diagnóstico al dataframe archivos_df

In [ ]:
archivos_df['Id'] = archivos_df['Id'].astype('float64') # se hace el cambio del tipo de variable A float64
archivos_df = pd.merge(archivos_df, diagnosticos, on='Id') # se unen los dos dataframes
archivos_df.head() # se revisa el encabezado

In [ ]:
#Esta Linea de código se usa para guardar el archivo en formato .zip
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
archivos_df.to_csv('out.zip', index=False,
          compression=compression_opts)
# El archivo de salida lo puedeencontrar en la carpeta Anexo 4

Se crean las sub carpetas de los diagnósticos

In [ ]:
# ds hace referencia a los posibles diagnósticos
for d in ds:
    path = os.path.join('salida2', d)
    os.makedirs(path)

# Se dividen los archivos

Para dividir los archivos se reccorre las caracteristicas de las anotaciones, donde estan el inicio y final de cada ciclo respiratorio, el diagnostico correspondiente y el nombre del archivo

In [ ]:
i = 0 #Se pone en cero para que al cambiar vaya nombrando el archivo

for idx, row in archivos_df.iterrows():
    inicio = row['inicio']
    fin = row['fin']
    filename = row['archivo']
    diag = row['diagnostico']
#Aca revisa la longitud del archivo y lo acota a 6s 
    if max_lon < fin - inicio:
        fin = inicio + max_lon
    audio_dir = audio_text_loc + '/' + f + '.wav'
    if idx != 0:
        if archivos_df.iloc[idx-1]['archivo'] == filename:
            i=i+1
        else:
            i=0
    n_filename = filename + '_' + str(i) + '.wav'
    path = 'salida2/' + diag + '/' + n_filename
    print('procesando audio ' + n_filename + '...')

    data, samplingrate = lb.load(audio_dir)
    audio_div = dividir_audio(start=inicio, end=fin, raw_data=data, sample_rate=samplingrate)
    a_len = compute_len(samp_rate=samplingrate, acquisition_mode=row['modo_adquisicion']=='sc')
    padded_data = lb.util.pad_center(audio_div, a_len)
    sf.write(file=path, data=padded_data, samplerate=samplingrate)